<a href="https://colab.research.google.com/github/vivekvijay107/Ai_Movie_Recommendation/blob/main/appp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#app_ai_movie_rec

In [12]:
!pip install streamlit pyngrok --quiet


In [13]:
from google.colab import files
uploaded = files.upload()


Saving tmdb_5000_movies.csv to tmdb_5000_movies (1).csv


In [14]:
%%writefile app.py
import streamlit as st
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load dataset
@st.cache_data
def load_data():
    df = pd.read_csv("tmdb_5000_movies.csv")
    df['overview'] = df['overview'].fillna('')
    return df

movies = load_data()

# TF-IDF matrix for overview
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['overview'])

# Mood matching
def get_mood_matches(mood, top_n=100):
    if not mood:
        return movies.index
    mood_vec = tfidf.transform([mood])
    similarity = cosine_similarity(mood_vec, tfidf_matrix).flatten()
    top_indices = similarity.argsort()[::-1][:top_n]
    return top_indices

# Recommendation logic
def suggest_movies(genre, year_pref, language, mood):
    df = movies.copy()
    mood_indices = get_mood_matches(mood)
    df = df.iloc[mood_indices]
    if genre:
        df = df[df['genres'].str.contains(genre, case=False, na=False)]
    if year_pref == "Recent":
        df = df[df['release_date'] >= '2015-01-01']
    if language:
        df = df[df['original_language'] == language]
    return df[['title', 'vote_average']].sort_values(by='vote_average', ascending=False).head(5)

# Streamlit UI
st.set_page_config(page_title="AI Movie Recommender", layout="centered")
st.title("🎬 AI Movie Recommender")
st.markdown("Get movie recommendations based on your **genre**, **language**, **release type**, and even your **mood** 🎭")

genre = st.text_input("🎞️ Genre (e.g., Action, Comedy, Drama)")
year_pref = st.selectbox("🕰️ Release Type", ["Recent", "Classic"])
language = st.text_input("🗣️ Language (e.g., en, hi, fr)")
mood = st.text_input("😄 Mood (e.g., happy, thrilling, romantic)")

if st.button("🎯 Recommend"):
    recommendations = suggest_movies(genre, year_pref, language, mood)
    if recommendations.empty:
        st.warning("No movies found matching your preferences.")
    else:
        st.success("Top Movie Recommendations:")
        for _, row in recommendations.iterrows():
            st.markdown(f"- **{row['title']}** — ⭐ {row['vote_average']}/10")


Overwriting app.py


In [15]:
!ngrok config add-authtoken 2zVdvqhZORegABQjSwRTOist41T_32ePwsqssb78LrapVDADe


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [17]:
from pyngrok import ngrok

# Start Streamlit app in the background
!streamlit run app.py &>/content/log.txt &

# ✅ FIX: Create public link correctly
public_url = ngrok.connect("http://localhost:8501")
print("🔗 Your Streamlit App is running at:", public_url)


🔗 Your Streamlit App is running at: NgrokTunnel: "https://171b-34-90-22-68.ngrok-free.app" -> "http://localhost:8501"
